In [1]:
import torch
import numpy as np
import pandas as pd
import glob
import os
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.io import read_image 
from torchvision import datasets, models, transforms
base_dir = '../dataset/MN_015_HH_01/MN_015_HH_01/handheld0/'

In [2]:
import torchvision.models as models
vgg = models.vgg16(pretrained=True)


c:\Users\28340\anaconda3\envs\fyp\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\28340\anaconda3\envs\fyp\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
df = pd.read_csv((base_dir+"cam0/"+"data.csv"))


In [4]:
#combine all possible 3 touple, 2 ref 1 query

In [5]:
class OivioDataset(Dataset):
    def __init__(self,annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)['filename']
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels[idx])
        image = read_image(img_path)
        label = self.img_labels[idx]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

    


In [6]:
dataset = OivioDataset("../dataset/MN_015_HH_01/MN_015_HH_01/handheld0/cam0/data.csv",base_dir+"cam0/data",transform=transforms.Resize([224,224]), target_transform=None)

In [7]:
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=4)

In [8]:
# del vgg.classifier
vgg.avgpool = torch.nn.Identity()
vgg.features[30] = torch.nn.Identity()
# print(vgg)

In [9]:
for param in vgg.features:
    param.requires_grad = False

In [15]:
class Upsampling(torch.nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=0):
        super(Upsampling, self).__init__()
        # print("in_channels: ",in_channels)
        # print("out_channels: ",out_channels)
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.upsample = torch.nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
                                     
    def forward(self, x1,x2,final = False):
        x1up = self.upsample(x1)
        # print("x1up?: ",x1up.shape)
        # print("x2.shape: ",x2.shape)
        if x1up.shape != x2.shape:
            x1up = torch.nn.functional.pad(x1up, (0, x2.shape[3] - x1up.shape[3], 0, x2.shape[2] - x1up.shape[2]))

        x = torch.cat([x2, x1up], dim=1)
        if final:
            output= torch.nn.Sequential(torch.nn.Conv2d(64+17, 17, kernel_size=1 , padding = 0,stride=1),
                                     torch.nn.BatchNorm2d(17),
                                     torch.nn.ReLU(True),
                                     
                                     torch.nn.Conv2d(17,17, kernel_size=1, padding= 0,stride=1),
                                     torch.nn.Sigmoid())(x)
            return output
        output = torch.nn.Sequential(torch.nn.Conv2d(self.in_channels, self.out_channels, kernel_size= 1, padding= 0,stride=1),
                                     torch.nn.BatchNorm2d(self.out_channels),
                                     torch.nn.ReLU(True),
                                     torch.nn.Conv2d(self.out_channels, self.out_channels, kernel_size= 1, padding= 0,stride=1),
                                     torch.nn.BatchNorm2d(self.out_channels),
                                     torch.nn.ReLU(True),
                                     torch.nn.Conv2d(self.out_channels, self.out_channels, kernel_size= 1, padding= 0,stride=1),
                                     torch.nn.BatchNorm2d(self.out_channels),
                                     torch.nn.ReLU(True))(x)
        

        return output

In [16]:
class UNet(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels

        self.vgg = vgg

        self.up_1 = Upsampling(1024, 17)
        self.up_2 = Upsampling(256+17, 17)
        self.up_3 = Upsampling(128+17, 17)
        self.up_4 = Upsampling(64+17, 17)
        # self.up_4 =
        # self.outputs = LastConv(64+17, 17)
    
    def forward(self,x):
        x1 = self.vgg.features[0:4](x)
        x2 = self.vgg.features[4:9](x1)
        x3 = self.vgg.features[9:16](x2)
        x4 = self.vgg.features[16:23](x3)
        x5 = self.vgg.features[23:30](x4)
        # print(x1.shape)
        # print(x2.shape)
        # print(x3.shape)
        # print(x4.shape)
        # print(x5.shape)

        x6 = self.up_1(x5,x4)
        x7 = self.up_2(x6,x3)
        x8 = self.up_3(x7,x2)
        x9 = self.up_4(x8,x1,True)
        print("x9.shape: ",x9.shape)
        # (F3,S3),(F2,S2),(F1,S1),(F0,S0)
        return (x6[:15],x6[15:]),(x7[:15],x7[15:]),(x8[:15],x8[16:]),(x9[:15],x9[16:])
        


In [17]:
net = UNet(1,17)

In [18]:
# for (x,y) in dataloader:
#     print(x.shape)
#     print(y.shape)
# print(dataset[0][0].)
(F3,S3),(F2,S2),(F1,S1),(F0,S0) = net(dataset[0][0].float().repeat(3,1,1).unsqueeze(0))

x9.shape:  torch.Size([1, 17, 224, 224])


In [14]:
dataset[0][0].shape

torch.Size([1, 224, 224])

In [19]:
S0.shape

torch.Size([0, 17, 223, 223])

In [30]:
import glob
import numpy as np
import argparse 

import torch
import torch.nn as nn
import torch.nn.functional as func
 
# from .code.models.LeastSquareTracking import LeastSquareTracking 
from tqdm import tqdm
from torch.utils.data import DataLoader
# from data.SimpleLoader import SimpleLoader
import numpy as np
import os
from numpy.linalg import inv
from numpy.matlib import repmat
import glob
import numpy as np
class Carla_Dataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, transform=None, target_transform=None):
        self.img_labels = glob.glob(os.path.join(img_dir, '*.npz'))
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.extrinsic_list = []
        self.intrinsic = 0
    def __len__(self):
        return len(self.img_labels)
    def find_depth(self, depth, intrinsic):
        depth_image = depth.detach().numpy()

        pixel_length = depth_image.shape[0] * depth_image.shape[1]
        width = depth_image.shape[1]
        height = depth_image.shape[0]
 
        u_coord = repmat(np.r_[width-1:-1:-1],
                        height, 1).reshape(pixel_length)
        v_coord = repmat(np.c_[height-1:-1:-1],
                        1, width).reshape(pixel_length)
        p2d = np.array([u_coord, v_coord, np.ones_like(u_coord)])

        normalized_depth = np.reshape(depth_image, pixel_length)
        p2d = np.array([u_coord, v_coord, np.ones_like(u_coord)])

        # P = [X,Y,Z]
        inv_intrinsic = inv(intrinsic)
        p3d = np.dot(inv_intrinsic, p2d)
        p3d = p3d * normalized_depth * 1000.0
        
        return p3d.reshape(height, width)

    def __getitem__(self, idx):
        # img_path = os.path.join(self.img_dir, self.img_labels[idx])
        img_path = self.img_labels[idx]
        image_info = np.load(img_path)
        rgb_image = image_info['rgb']
        depth_image = image_info['depth']
        # depth = findDepth(depth_image, intrinsic)
        depth = depth_image
        intrinsic = image_info['K']
        extrinsic = image_info['extrinsic']
        self.intrinsic = torch.from_numpy(intrinsic)
        self.extrinsic_list.append(torch.from_numpy(extrinsic))
        

        if self.transform:
            rgb_image = self.transform(rgb_image)
            depth_image = self.transform(depth_image)
            depth_image = self.find_depth(depth_image, intrinsic)
        return torch.from_numpy(rgb_image).type(torch.float).view(3, depth_image.shape[0], depth_image.shape[1]).unsqueeze(0), torch.from_numpy(depth_image).type(torch.float).view(1, depth_image.shape[0], depth_image.shape[1]).unsqueeze(0)
    

def resize(img0, img1, depth0, depth1, resizeH, resizeW): 
    H, W = img0.shape[-2:]
    
    I0 = func.interpolate(img0, (resizeH,resizeW), mode='bilinear', align_corners=True)
    I1 = func.interpolate(img1, (resizeH,resizeW), mode='bilinear', align_corners=True)
    D0 = func.interpolate(depth0, (resizeH,resizeW), mode='nearest')
    D1 = func.interpolate(depth1, (resizeH,resizeW), mode='nearest')

    sx = resizeH / H
    sy = resizeW / W

    return I0, I1, D0, D1



In [31]:
dataset = Carla_Dataset(f'C:\\Users\\28340\\Documents\\UCL\\finalYearProject\\code\\unsupervised_vis_loc_reproduce\\carla_npzs\\')
simple_loader = DataLoader(dataset, batch_size=1, shuffle=False)
temp_image = dataset[0]

In [32]:
color0,depth0 = dataset[0]

color1,depth1 = dataset[1]
extrinsic1 = dataset.extrinsic_list[0]
extrinsic2 = dataset.extrinsic_list[1] 
C0, C1, D0, D1 = resize(color0, color1, depth0, depth1, resizeH = 120, resizeW=160)
# with torch.no_grad():
result= net.forward(C0)

x9.shape:  torch.Size([1, 17, 120, 160])


In [34]:
(F3,S3),(F2,S2),(F1,S1),(F0,S0) = result

In [36]:
F3.shape

torch.Size([1, 17, 15, 20])

In [37]:
F2.shape

torch.Size([1, 17, 30, 40])

In [38]:
F1.shape

torch.Size([1, 17, 60, 80])

In [39]:
F0.shape

torch.Size([1, 17, 120, 160])